In [1]:
from car_racing import CarRacing

import scipy.misc
import imageio
import numpy as np
import gym

import msvcrt
import win32api
import win32con

from multiprocess import Process, Queue, Manager
import os

from random import choice, random, randint

env = CarRacing()


D:\Users\Kristopher_2\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
#Code written by Tielessin on https://stackoverflow.com/questions/46506850/how-can-i-get-input-from-an-xbox-one-controller-in-python
#Edited by Kristopher Karadimas

from inputs import get_gamepad
import math
import threading

class XboxController(object):
    MAX_TRIG_VAL = math.pow(2, 8)
    MAX_JOY_VAL = math.pow(2, 15)

    def __init__(self):

        self.LeftJoystickY = 0
        self.LeftJoystickX = 0
        self.RightJoystickY = 0
        self.RightJoystickX = 0
        self.LeftTrigger = 0
        self.RightTrigger = 0
        self.LeftBumper = 0
        self.RightBumper = 0
        self.A = 0
        self.X = 0
        self.Y = 0
        self.B = 0
        self.LeftThumb = 0
        self.RightThumb = 0
        self.Back = 0
        self.Start = 0
        self.LeftDPad = 0
        self.RightDPad = 0
        self.UpDPad = 0
        self.DownDPad = 0

        self._monitor_thread = threading.Thread(target=self._monitor_controller, args=())
        self._monitor_thread.daemon = True
        self._monitor_thread.start()


    def read(self): # return the buttons/triggers that you care about in this methode
        x = self.LeftJoystickX
        rt = self.RightTrigger
        lt = self.LeftTrigger
        return [x, rt, lt]


    def _monitor_controller(self):
        while True:
            events = get_gamepad()
            for event in events:
                if event.code == 'ABS_Y':
                    self.LeftJoystickY = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_X':
                    self.LeftJoystickX = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_RY':
                    self.RightJoystickY = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_RX':
                    self.RightJoystickX = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_Z':
                    self.LeftTrigger = event.state / XboxController.MAX_TRIG_VAL # normalize between 0 and 1
                elif event.code == 'ABS_RZ':
                    self.RightTrigger = event.state / XboxController.MAX_TRIG_VAL # normalize between 0 and 1
                elif event.code == 'BTN_TL':
                    self.LeftBumper = event.state
                elif event.code == 'BTN_TR':
                    self.RightBumper = event.state
                elif event.code == 'BTN_SOUTH':
                    self.A = event.state
                elif event.code == 'BTN_NORTH':
                    self.X = event.state
                elif event.code == 'BTN_WEST':
                    self.Y = event.state
                elif event.code == 'BTN_EAST':
                    self.B = event.state
                elif event.code == 'BTN_THUMBL':
                    self.LeftThumb = event.state
                elif event.code == 'BTN_THUMBR':
                    self.RightThumb = event.state
                elif event.code == 'BTN_SELECT':
                    self.Back = event.state
                elif event.code == 'BTN_START':
                    self.Start = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY1':
                    self.LeftDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY2':
                    self.RightDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY3':
                    self.UpDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY4':
                    self.DownDPad = event.state





In [3]:
joy = XboxController()

def get_action():
        action = env.action_space.sample()
        inputs = joy.read()
        
        joystickinputfilter = 0 if (abs(inputs[0]) < 0.15) else inputs[0]
        joystickinputsign = -1 if (joystickinputfilter<0) else 1
        action[0] = joystickinputsign*abs((joystickinputfilter*0.65)**3)
        action[1] = min(inputs[1],0.8)**6
        action[2] = min(inputs[2],0.8)**6
        return action

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
bs = 32

# Device configuration
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [5]:
#Neural Network Parameters
num_epochs = 30
batch_size = bs
learning_rate = 0.001

input_size = 64 #Dimensions of the input "concatenatedTensor"
hidden_size = [32,16,8]
num_classes = 3 #Three state outputs. Acceleration. Braking. Turning Angle

#Define neural network for this part
class StateDetectionNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(StateDetectionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1]) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2]) 
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size[2], num_classes)  
    
    def forward(self, x):
        out = self.fc1(x.reshape(-1,input_size))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

model2 = StateDetectionNet(input_size, hidden_size, num_classes).to(device)
model2.load_state_dict(torch.load('CurrentConditions.torch', map_location=device))

<All keys matched successfully>

In [6]:
class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        esp = esp.to(device)
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar
    
image_channels = 3
model = VAE(image_channels=image_channels).to(device)
model.load_state_dict(torch.load('vae.torch', map_location=device))
model=model.to(device)

In [7]:
directory = f'D:/Kris\'s Workbench/FYP/TrainingData/0/'

import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image






obs = env.reset()
env.render()



t = 0
r = 0

#while(True):
action = get_action()
obs, reward, done, _ = env.step(action)
env.render()
PreviousImage = obs
CurrentImage = obs

for t in range(5000):
    t+=1
    action = get_action()
    obs, reward, done, _ = env.step(action)
    env.render()
    r += reward
    
    eps=1
    action_0 = action[0]
    action_1 = action[1]
    action_2 = action[2]
    
    PreviousImage = CurrentImage
    CurrentImage = obs
    
    
    im = Image.fromarray(np.uint8(PreviousImage)).convert('RGB')
    im2 = Image.fromarray(np.uint8(CurrentImage)).convert('RGB')
    
    #CurrentImage = torch.from_numpy(np.flip(CurrentImage,axis=0).copy())
    #PreviousImage = torch.from_numpy(np.flip(PreviousImage,axis=0).copy())
    
    preprocess=transforms.Compose([
            transforms.Resize(64),
            transforms.ToTensor(), 
            ])
        
    im_preprocessed = preprocess(im2).to(device)
    previousim_preprocessed = preprocess(im).to(device)
    testCurrent = torch.unsqueeze(im_preprocessed, 0).to(device)
    testPrevious = torch.unsqueeze(previousim_preprocessed, 0).to(device)
    



    
    curr, _, _ = model.encode(testCurrent)
    prev, _, _ = model.encode(testPrevious)
    concatenatedTensor = torch.cat([prev, curr], dim=1)
    
    
    from IPython.display import clear_output
    out = model2(concatenatedTensor).cpu().detach().numpy() 
    clear_output(wait=True)
    
    mean_col1 = -0.026595786253976014
    mean_col2 = 0.21783560955090628
    mean_col3 = 0.007642476004356518
    sd_col1 = 0.10711904839787884
    sd_col2 = 0.09744389996469334
    sd_col3 = 0.043793150114235936
    col1 = (out[:,0]*sd_col1)+mean_col1
    col2 = (out[:,1]*sd_col2)+mean_col2
    col3 = (out[:,2]*sd_col3)+mean_col3
    np.set_printoptions(precision=2)
    out = np.column_stack((col1,col2,col3))
    print(out)
    
env.close()

[[-0.01  0.29 -0.  ]]


In [7]:
directory = f'D:/Kris\'s Workbench/FYP/TrainingData/0/'

import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image






obs = env.reset()
env.render()



t = 0
r = 0

#while(True):
action = get_action()
obs, reward, done, _ = env.step(action)
env.render()
PreviousImage = obs
CurrentImage = obs

for t in range(1000):
    t+=1

    
    PreviousImage = CurrentImage
    CurrentImage = obs
    
    
    im = Image.fromarray(np.uint8(PreviousImage)).convert('RGB')
    im2 = Image.fromarray(np.uint8(CurrentImage)).convert('RGB')
    
    #CurrentImage = torch.from_numpy(np.flip(CurrentImage,axis=0).copy())
    #PreviousImage = torch.from_numpy(np.flip(PreviousImage,axis=0).copy())
    
    preprocess=transforms.Compose([
            transforms.Resize(64),
            transforms.ToTensor(), 
            ])
        
    im_preprocessed = preprocess(im2).to(device)
    previousim_preprocessed = preprocess(im).to(device)
    testCurrent = torch.unsqueeze(im_preprocessed, 0).to(device)
    testPrevious = torch.unsqueeze(previousim_preprocessed, 0).to(device)
    



    
    curr, _, _ = model.encode(testCurrent)
    prev, _, _ = model.encode(testPrevious)
    concatenatedTensor = torch.cat([prev, curr], dim=1)
    
    
    from IPython.display import clear_output
    out = model2(concatenatedTensor).cpu().detach().numpy() 
    clear_output(wait=True)
    
    mean_col1 = -0.026595786253976014
    mean_col2 = 0.21783560955090628
    mean_col3 = 0.007642476004356518
    sd_col1 = 0.10711904839787884
    sd_col2 = 0.09744389996469334
    sd_col3 = 0.043793150114235936
    col1 = (out[:,0]*sd_col1)+mean_col1
    col2 = (out[:,1]*sd_col2)+mean_col2
    col3 = (out[:,2]*sd_col3)+mean_col3
    np.set_printoptions(precision=2)
    out = np.column_stack((col1,col2,col3))
    print(out)
    
    action = get_action()
    action[0]=col1
    action[1]=col2
    action[2]=col3
    obs, reward, done, _ = env.step(action)
    env.render()
    r += reward
    
    eps=1
    action_0 = action[0]
    action_1 = action[1]
    action_2 = action[2]
    
env.close()

[[-0.   -0.01  0.  ]]


In [8]:
directory = f'D:/Kris\'s Workbench/FYP/TrainingData/0/'

import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image






obs = env.reset()
env.render()



t = 0
r = 0

#while(True):
action = get_action()
obs, reward, done, _ = env.step(action)
env.render()
PreviousImage = obs
CurrentImage = obs

for t in range(500):
    t+=1
    action = get_action()
    obs, reward, done, _ = env.step(action)
    env.render()
    r += reward
    
    eps=1
    action_0 = action[0]
    action_1 = action[1]
    action_2 = action[2]
    
    PreviousImage = CurrentImage
    CurrentImage = obs
    
    
    im = Image.fromarray(np.uint8(PreviousImage)).convert('RGB')
    im2 = Image.fromarray(np.uint8(CurrentImage)).convert('RGB')
    
    #CurrentImage = torch.from_numpy(np.flip(CurrentImage,axis=0).copy())
    #PreviousImage = torch.from_numpy(np.flip(PreviousImage,axis=0).copy())
    
    preprocess=transforms.Compose([
            transforms.Resize(64),
            transforms.ToTensor(), 
            ])
        
    im_preprocessed = preprocess(im2).to(device)
    previousim_preprocessed = preprocess(im).to(device)
    testCurrent = torch.unsqueeze(im_preprocessed, 0).to(device)
    testPrevious = torch.unsqueeze(previousim_preprocessed, 0).to(device)
    



    
    curr, _, _ = model.encode(testCurrent)
    prev, _, _ = model.encode(testPrevious)
    concatenatedTensor = torch.cat([prev, curr], dim=1)
    
    
    from IPython.display import clear_output
    out = model2(concatenatedTensor).cpu().detach().numpy() 
    clear_output(wait=True)
    
    mean_col1 = -0.026595786253976014
    mean_col2 = 0.21783560955090628
    mean_col3 = 0.007642476004356518
    sd_col1 = 0.10711904839787884
    sd_col2 = 0.09744389996469334
    sd_col3 = 0.043793150114235936
    col1 = (out[:,0]*sd_col1)+mean_col1
    col2 = (out[:,1]*sd_col2)+mean_col2
    col3 = (out[:,2]*sd_col3)+mean_col3
    np.set_printoptions(precision=2)
    out = np.column_stack((col1,col2,col3))
    print(out)
    
env.close()

[[-0.    0.25  0.  ]]
